# <font color=blue>Project Name : HT_and_OneApp_Recom</font>

**Notebook**: OneApp and HTRecom<br>

**Title**: It generates Oneapp recommendation and HT recommendations.<br>

**Short summary**: This notebook extracts NPS values and convert them into recommendation based on some conditions,<br>


**Analyst**: Aman NAGAR<br>

**Duration** : December <br>

**Version**: 0.1<br>



In [ ]:
%%configure -f
{
"kind": "pyspark",

"conf": {
   
     "spark.master": "yarn",
    "spark.yarn.dist.archives": "/use_case/hhmatching/Library_zip/extra_lib_3.zip#lcm_zip", 
    "spark.yarn.appMasterEnv.PYSPARK_PYTHON": "./lcm_zip/bin/python",
    "spark.driver.cores": "8", 
    "spark.driver.memory": "13000m",  
    "spark.executor.memory": "13000m",
    "spark.executor.cores": "8", 
     "spark.dynamicAllocation.enabled": "true",
     "spark.dynamicAllocation.minExecutors": "3"
    
      
    }
}

### Import Libraries<a name="libs"></a>
<a href = "#toc">Back to top</a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import dataframe
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import lit,log
from pyspark.sql.functions import isnan
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql.functions import col, lit, coalesce, greatest, concat
from pyspark.sql.functions import regexp_replace 
from pyspark.sql import DataFrame
from pyspark.sql.window import *
from pyspark.sql.functions import row_number
from typing import Iterable
import pandas as pd
import numpy as np
from pyspark.sql.functions import unix_timestamp, from_unixtime,hour
from pyspark.sql.types import StringType,FloatType,DoubleType,LongType,IntegerType,ArrayType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import Bucketizer
import pandas as pd, numpy as np
from matplotlib import pyplot as plt
#from plotly.offline import iplot
#import plotly.graph_objs as go
#%matplotlib inline
import seaborn as sns
from pandas.plotting import table
#from sklearn.preprocessing import LabelEncoder
import matplotlib.patches as mpatches
import datetime
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder 
#from sklearn.metrics import roc_curve, auc
#from sklearn.metrics import log_loss
from pyspark.sql import Window
from pyspark.sql.functions import rank,sum,col
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorSlicer

from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.sql.functions import when
from pyspark.sql import functions as F
from pyspark.sql.functions import avg
from pyspark.ml import Pipeline
# Initiate Spark Session
# spark = SparkSession\
#     .builder\
#     .appName("example-spark")\
#     .config("spark.sql.crossJoin.enabled","true")\
#     .getOrCreate()

# ## Creating Hive Context

# sc = pyspark.SparkContext()
# hiveContext = HiveContext(sc)

#spark = SparkSession.builder\
# .master("local[2]")\
# .config("spark.submit.deployMode", "client")\
# .config("spark.eventLog.enabled", "true")\
# .config("spark.dynamicAllocation.enabled", "true")\
# .config("spark.dynamicAllocation.executorIdleTimeout", "1500s")\
# .config("spark.dynamicAllocation.initialExecutors", "53")\
# .appName("Base_Line")\
# .getOrCreate()
#
#sc = spark.sparkContext
#L = spark.sparkContext._conf.getAll()

#bi_reason = sys.argv[0]

#conf = SparkConf().setAppName('Base_Line')
#sc = SparkContext(conf=conf)

#sqlContext = HiveContext(sc)

spark.conf.set("spark.sql.execution.arrow.enabled","false")

spark.conf.set("spark.sql.execution.arrow.fallback.enabled","true")

from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

from pyspark.sql import HiveContext
hiveContext = HiveContext(spark)


In [ ]:
def dataImport(db,table):
    df = spark.sql("select * from {0}.{1}".format(db,table))
    return df

In [ ]:
df = dataImport("cdl_tdata_model","oneappusageevents")

In [ ]:
df = df.select("oneapp_user_id", "event_params.ATTR_value_NUM","event_params.ATTR_label_STR","event_name",'event_timestamp')

In [ ]:
nps_data = df.select("oneapp_user_id", "ATTR_value_NUM","ATTR_label_STR","event_name","event_timestamp").filter(df.event_name == "InApp_Score_Submitted")
# count_users = nps_data.select("oneapp_user_id").count()

In [ ]:
# df3 = df2.withColumn("parsed", F.to_timestamp("date", "yyyy-MM-dd HH:mm:ss"))

In [ ]:
nps_data = nps_data.withColumn("sept1",F.lit("2020-12-1 00:00:00"))
nps_data = nps_data.withColumn("sept2",F.lit("2020-12-31 00:00:00"))
# nps_data = nps_data.withColumn("oct2",F.lit("2020-10-31 00:00:00"))

In [ ]:
nps_data = nps_data.withColumn("sept1", F.to_timestamp("sept1", "yyyy-MM-dd HH:mm:ss"))
nps_data = nps_data.withColumn("sept2", F.to_timestamp("sept2", "yyyy-MM-dd HH:mm:ss"))
# nps_data = nps_data.withColumn("oct2", F.to_timestamp("oct2", "yyyy-MM-dd HH:mm:ss"))


In [ ]:
nps_data = nps_data.withColumn("event_timestamp", F.to_timestamp("event_timestamp", "yyyy-MM-dd HH:mm:ss"))

In [ ]:
nps_data

In [ ]:
nps_data1 = nps_data.filter((F.col("event_timestamp") >= F.col("sept1")) & (F.col("event_timestamp") <= F.col("sept2")))

In [ ]:
# nps_data2 = nps_data.filter((F.col("event_timestamp") > F.col("sept2")) & (F.col("event_timestamp") <= F.col("oct2")))

In [ ]:
nps_data_sept_ht = nps_data1.filter(nps_data1.ATTR_label_STR == "NPS - Hrvatski Telekom")

In [ ]:
# nps_data_oct_ht = nps_data2.filter(nps_data1.ATTR_label_STR == "NPS - Hrvatski Telekom")

In [ ]:
nps_data_sept_op = nps_data1.filter(nps_data1.ATTR_label_STR == "NPS - Moj Telekom aplikacija")

In [ ]:
# nps_data_oct_op = nps_data2.filter(nps_data1.ATTR_label_STR == "NPS - Moj Telekom aplikacija")

# Save Intermediate Results

In [ ]:
nps_data_sept_ht.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.nps_data_dec_ht as select * from myTemp11")

In [ ]:
# nps_data_oct_ht.createOrReplaceTempView("myTemp11")
# sqlContext.sql("create table cdl_common.nps_data_oct_ht as select * from myTemp11")

In [ ]:
nps_data_sept_op.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.nps_data_dec_op as select * from myTemp11")

In [ ]:
# nps_data_oct_op.createOrReplaceTempView("myTemp11")
# sqlContext.sql("create table cdl_common.nps_data_oct_op as select * from myTemp11")

# HT Recommedations

In [ ]:
nps_data_sept_ht1 = dataImport("cdl_common","nps_data_dec_ht")

In [ ]:
w = Window.partitionBy('oneapp_user_id').orderBy(F.desc('event_timestamp'))
nps_data_sept_ht1 = nps_data_sept_ht1.select('*', row_number().over(w).alias("Rank"))

nps_data_sept_ht1 = nps_data_sept_ht1.filter(nps_data_sept_ht1.Rank == 1)

In [ ]:
nps_data_sept_ht1 = nps_data_sept_ht1.drop_duplicates(subset=['oneapp_user_id'])

In [ ]:
nps_data_sept_ht1 = nps_data_sept_ht1.withColumn("ATTR_value_NUM",F.col("ATTR_value_NUM").cast(IntegerType()))

In [ ]:
nps_data_sept_ht1

In [ ]:
nps_data_sept_ht1 = nps_data_sept_ht1.filter(F.col("ATTR_value_NUM") != 7)
nps_data_sept_ht1 = nps_data_sept_ht1.filter(F.col("ATTR_value_NUM") != 8)

In [ ]:
nps_data_sept_ht1 = nps_data_sept_ht1.withColumn("HT_recommended_value",when(F.col("attr_value_num") > 7 ,F.lit(0)).otherwise(F.lit(1)))

In [ ]:
nps_data_sept_ht1

In [ ]:
nps_data_sept_ht1 = nps_data_sept_ht1.select("oneapp_user_id","ATTR_value_NUM","event_timestamp","HT_recommended_value")

In [ ]:
nps_data_sept_ht1.count()

In [ ]:
nps_data_sept_ht1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.HT_recommended_value_dec as select * from myTemp11")

# OneApp Recommendation

In [ ]:
nps_data_sept_op1 = dataImport("cdl_common","nps_data_dec_op")

In [ ]:
w = Window.partitionBy('oneapp_user_id').orderBy(F.desc('event_timestamp'))
nps_data_sept_op1 = nps_data_sept_op1.select('*', row_number().over(w).alias("Rank"))

nps_data_sept_op1 = nps_data_sept_op1.filter(nps_data_sept_op1.Rank == 1)

In [ ]:
nps_data_sept_op1 = nps_data_sept_op1.drop_duplicates(subset=['oneapp_user_id'])

In [ ]:
nps_data_sept_op1 = nps_data_sept_op1.withColumn("ATTR_value_NUM",F.col("ATTR_value_NUM").cast(IntegerType()))

In [ ]:
nps_data_sept_op1

In [ ]:
nps_data_sept_op1 = nps_data_sept_op1.filter(F.col("ATTR_value_NUM") != 7)
nps_data_sept_op1 = nps_data_sept_op1.filter(F.col("ATTR_value_NUM") != 8)

In [ ]:
nps_data_sept_op1 = nps_data_sept_op1.withColumn("OneApp_recommended_value",when(F.col("attr_value_num") > 7 ,F.lit(0)).otherwise(F.lit(1)))

In [ ]:
nps_data_sept_op1

In [ ]:
nps_data_sept_op1 = nps_data_sept_op1.select("oneapp_user_id","ATTR_value_NUM","event_timestamp","OneApp_recommended_value")

In [ ]:
nps_data_sept_op1.count()

In [ ]:
nps_data_sept_op1.createOrReplaceTempView("myTemp11")
sqlContext.sql("create table cdl_common.OA_recommended_value_dec as select * from myTemp11")